Notebook for creating the Peptide dataset

In [43]:
import pandas as pd
import torch
import numpy as np
from peptidevae.load_vae import load_vae, vae_forward, vae_decode
from tqdm import tqdm
import pickle
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# This takes a while
vae, dataobj = load_vae("peptidevae/checkpoints/dim128_k1_kl0001_eff256_dff256_pious-sea-2_model_state_epoch_118.pkl", dim=256, max_string_length=50)

c:\Users\2023r\Documents\GuidedDiffusionProject\peptidevae\load_vae.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  


In [ ]:
# Test VAE is workring
test_seqs = ["RWWLKMMLRMRI", "WKLKIRWRLRFLRLWFLWLWWWWFWLWRWK"]
with torch.no_grad():
    latents = vae_forward(test_seqs, dataobj, vae)

# two latents of shape 256
latents[0].to('cpu').numpy().shape

c:\Users\2023r\Documents\GuidedDiffusionProject\.venv\Lib\site-packages\torch\nn\modules\transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(


(2, 256)

In [ ]:
PEPTIDE_10M_LABELED = "data/Extinct_labels_10M.csv"
peptide_10m_df = pd.read_csv(PEPTIDE_10M_LABELED)
sequences, labels = peptide_10m_df['text'], peptide_10m_df['labels']


In [ ]:

# Takes in the (peptide_seq, labels) dataset and creates a (latent, labels) dataset with the vae
def create_latent_dataset(sequences, labels, vae=vae, dataobj=dataobj, batch_size=256, output_path="data/latent_dataset.pkl", device=device):
    '''
        sequences: list of peptide sequences
        labels: list of labels
        vae, dataobj: the VAE we are usign to forward
    '''

    all_latents = []

    vae.eval()
    with torch.no_grad():
        for i in tqdm(range(0, len(sequences), batch_size)):
            batch_seqs = sequences[i:i+batch_size]

            latents, _ = vae_forward(batch_seqs, dataobj, vae)
            all_latents.append(latents.to('cpu'))
    
    all_latents = np.concatenate(all_latents, axis=0)
    dataset_dict = {
        'latent': all_latents,
        'labels': labels
    }


    with open(output_path, 'wb') as f:
        pickle.dump(dataset_dict, f)

    print(f"Saved {len(sequences)} peptide entries with sequences, latent codes, and labels to {output_path}")

    return dataset_dict



In [ ]:
# Create the whole latent dataset
# Should never need to run this again

# create_latent_dataset(sequences.to_numpy(), labels.to_numpy())

100%|██████████| 40136/40136 [21:56<00:00, 30.49it/s]   


Saved 10274723 peptide entries with sequences, latent codes, and labels to data/latent_dataset.pkl


{'latent': array([[-0.31667215,  0.14633523, -0.26224312, ...,  0.05510085,
         -0.3740702 , -0.38232166],
        [-0.07624465,  0.17372577, -0.14082399, ..., -0.05611906,
          0.00884153,  0.02623769],
        [ 0.01839117,  0.36293083, -0.08525313, ...,  0.32207313,
         -0.10469632,  0.04467239],
        ...,
        [ 0.35400948, -0.32883057, -0.08890741, ...,  0.07047714,
         -0.5032545 ,  0.41565666],
        [ 0.06051294, -0.08147617,  0.06925601, ..., -0.1951151 ,
         -0.08448125, -0.22564164],
        [-0.52381885, -0.30431655,  0.19177069, ...,  0.14462498,
         -0.0253647 , -0.11564276]], shape=(10274723, 256), dtype=float32),
 'labels': array([1, 0, 1, ..., 0, 1, 0], shape=(10274723,))}

In [ ]:
import importlib
import peptide_dataset
importlib.reload(peptide_dataset)

data_module = peptide_dataset.LatentDataModule()

# Should take like ~10 seconds w/ 10M examples

Using: cuda
Using: cuda
Dataset split: 9247250 training samples, 1027473 validation samples
